In [1]:
import torch
import numpy as np
from typing import Dict, List, Optional
import json
import os
from scipy.stats import zscore
from datetime import datetime
import uuid

from EEGPTPatientAssessment import EEGPTPatientAssessment

In [2]:
def analyze_patient_eeg(voltage_data: np.ndarray, sampling_rate: int = 250, 
                       channel_names: Optional[List[str]] = None) -> Dict[str, any]:
    """
    **MAIN FUNCTION: Analyze patient EEG and return interpretable assessment**
    
    Args:
        voltage_data: Raw EEG voltages in volts, shape (channels, timepoints)
        sampling_rate: Sampling frequency in Hz
        channel_names: Optional channel names
        
    Returns:
        Complete interpretable patient assessment
    """
    
    # Initialize assessment system
    assessor = EEGPTPatientAssessment()
    
    # Analyze EEG
    assessment = assessor.analyze_eeg(voltage_data, sampling_rate, channel_names)
    
    return assessment

In [ ]:
if __name__ == "__main__":
    # Example with synthetic EEG data
    print("EEGPT Patient Assessment System")
    print("=" * 50)
    
    # Generate sample EEG data (replace with your real data)
    np.random.seed(42)
    n_channels = 8
    n_timepoints = 1000  # 4 seconds at 250 Hz
    sampling_rate = 250
    
    # Create realistic EEG voltage patterns
    t = np.linspace(0, 4, n_timepoints)
    voltage_data = np.zeros((n_channels, n_timepoints))
    
    for i in range(n_channels):
        # Alpha rhythm (10 Hz) - relaxed state
        voltage_data[i] += 50e-6 * np.sin(2 * np.pi * 10 * t)
        # Beta activity (20 Hz) - alert state
        voltage_data[i] += 20e-6 * np.sin(2 * np.pi * 20 * t)
        # Add realistic noise
        voltage_data[i] += 10e-6 * np.random.randn(n_timepoints)
    
    channel_names = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2']
    
    # **ANALYZE EEG DATA**
    assessment = analyze_patient_eeg(voltage_data, sampling_rate, channel_names)
    
    # **DISPLAY RESULTS**
    print("\nPATIENT ASSESSMENT RESULTS:")
    print("-" * 30)
    
    summary = assessment['patient_summary']
    print(f"Overall State: {summary['overall_brain_state']}")
    print(f"Alertness: {summary['alertness_level']}")
    print(f"Stress Level: {summary['stress_level']}")
    print(f"Attention: {summary['attention_score']}")
    print(f"Cognitive State: {summary['cognitive_state']}")
    print(f"Seizure Risk: {summary['seizure_risk']}")
    print(f"Confidence: {summary['confidence_level']}")
    
    print(f"\nCLINICAL INTERPRETATIONS:")
    print("-" * 30)
    interpretations = assessment['clinical_interpretations']
    for key, value in interpretations.items():
        print(f"{key.title()}: {value}")
    
    print(f"\nRECOMMENDATIONS:")
    print("-" * 30)
    for i, rec in enumerate(assessment['recommendations'], 1):
        print(f"{i}. {rec}")
    
    print(f"\nMOST IMPORTANT CHANNELS:")
    print(f"{', '.join(assessment['interpretability']['most_important_channels'])}")
    
    # Save results
    with open('data/patient_eeg_assessment.json', 'w') as f:
        json.dump(assessment, f, indent=2)
    
    print(f"\n✓ Complete assessment saved to 'patient_eeg_assessment.json'")


EEGPT Patient Assessment System
Loading pretrained EEGPT model...
✓ Pretrained EEGPT model loaded successfully
✓ EEGPT Patient Assessment initialized on cpu
✓ Using pretrained model: checkpoint/eegpt_mcae_58chs_4s_large4E.ckpt
Analyzing EEG: (8, 1000) channels, 1000 samples
Duration: 4.0 seconds

PATIENT ASSESSMENT RESULTS:
------------------------------
Overall State: Normal Brain Activity
Alertness: 5.1/10
Stress Level: 5.2/10
Attention: 6.2/10
Cognitive State: Normal
Seizure Risk: Low Risk
Confidence: 88.0%

CLINICAL INTERPRETATIONS:
------------------------------
Alertness: Moderate alertness - Mild fatigue or reduced focus may be present
Stress: Moderate stress - Within normal range for daily activities
Attention: Good attention - Normal cognitive function and focus ability
Cognitive: Normal (confidence: 38.9%)
Seizure_Risk: Low Risk (confidence: 36.0%)

RECOMMENDATIONS:
------------------------------
1. Normal findings - routine monitoring sufficient

MOST IMPORTANT CHANNELS:
F3,